In [1]:
import tensorflow as tf

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Combine the training and test data into a single data variable
data = (x_train, y_train), (x_test, y_test)

In [2]:
import tensorflow as tf
import numpy as np

def scale_image(image, scale):
    return tf.image.resize(image, [int(image.shape[0] * scale), int(image.shape[1] * scale)])

def rotate_image(image, angle):
    return tf.image.rot90(image, k=angle)

def add_noise(image, noise_factor=0.5):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=noise_factor, dtype=tf.float32)
    return tf.clip_by_value(image + noise, 0.0, 1.0)

def augment_image(image):
    scale = np.random.uniform(0.8, 1.2)
    angle = np.random.randint(0, 4)
    noise_factor = np.random.uniform(0.1, 0.5)
    
    image = tf.expand_dims(image, axis=-1)  # Add channel dimension
    image = scale_image(image, scale)
    image = rotate_image(image, angle)
    image = add_noise(image, noise_factor)
    
    return image

original_shape = x_train.shape[1:3]

aug_train_x = np.array([tf.image.resize(augment_image(tf.convert_to_tensor(img, dtype=tf.float32)), original_shape).numpy() for img in x_train])
aug_test_x = np.array([tf.image.resize(augment_image(tf.convert_to_tensor(img, dtype=tf.float32)), original_shape).numpy() for img in x_test])

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Print the model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)               

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle

# Convert labels to categorical one-hot encoding
y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(aug_train_x, y_train_cat, epochs=10, validation_data=(aug_test_x, y_test_cat))

Epoch 1/10


1875/1875 [==============================] - 32s 15ms/step - loss: 0.6366 - accuracy: 0.7911 - val_loss: 0.3612 - val_accuracy: 0.8822
Epoch 2/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.2865 - accuracy: 0.9093 - val_loss: 0.2397 - val_accuracy: 0.9251
Epoch 3/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.2065 - accuracy: 0.9346 - val_loss: 0.2034 - val_accuracy: 0.9356
Epoch 4/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1626 - accuracy: 0.9481 - val_loss: 0.2136 - val_accuracy: 0.9332
Epoch 5/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1369 - accuracy: 0.9556 - val_loss: 0.1802 - val_accuracy: 0.9402
Epoch 6/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1153 - accuracy: 0.9628 - val_loss: 0.1834 - val_accuracy: 0.9430
Epoch 7/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0985 - accuracy: 0.9681 - val_loss: 0.1887

In [6]:
# Save the model in HDF5 format
model.save('model.h5')


c:\Users\singh\.conda\envs\DL\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from tensorflow.keras.models import load_model

# Load the saved model
saved_model = load_model('model.h5')

# Evaluate the model on the test data
loss, accuracy = saved_model.evaluate(aug_test_x, y_test_cat)
print(f'Test accuracy: {accuracy * 100:.2f}%')

313/313 [==============================] - 3s 7ms/step - loss: 0.2065 - accuracy: 0.9446
Test accuracy: 94.46%
